In [1]:
from urllib import request
from pathlib import Path
import argparse
import random
import torch
import gradio as gr
from gradio import ChatMessage
from PIL import Image, ImageOps
import requests
from io import BytesIO

from peft import AutoPeftModelForCausalLM

from transformers import AutoTokenizer, BitsAndBytesConfig
from time import sleep

c:\Users\rnltl\anaconda3\envs\gemma\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class HPChatBot:
    def __init__(self,
                 model_path: str = 'rnltls/harrypotter_lexicon_finetune',
                 device_map: str = 'auto',
                 load_in_4_bit: bool = True,
                 **quant_kwargs) -> None:
        self.model = None
        self.tokenizer = None
        self.image_processor = None
        self.conv = None
        self.conv_img = []
        self.img_tensor = []
        self.roles = None
        self.stop_key = None
        self.is_chat = False
        self.is_waldo = False
        self.load_models(model_path,
                         device_map=device_map,
                         load_in_4_bit=load_in_4_bit,
                         **quant_kwargs)

    def load_models(self, model_path: str,
                    device_map: str,
                    load_in_4_bit: bool,
                    **quant_kwargs) -> None:
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_path, # YOUR MODEL YOU USED FOR TRAINING
            load_in_4bit = load_in_4_bit,
        )
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        self.model = AutoPeftModelForCausalLM.from_pretrained(
            "rnltls/harrypotter_lexicon_finetune", # YOUR MODEL YOU USED FOR TRAINING
            load_in_4bit = load_in_4_bit,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_path,
                                                       use_fast=False)
    
    def generate_answer(self, prompt):
        output = self.model.generate(**prompt, max_new_tokens = 128)
        generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)
        splited_generated_text = generated_text.split("Response:")[-1]
        return splited_generated_text

HP_chatbot = HPChatBot(load_in_8bit=True,
                bnb_8bit_compute_dtype=torch.float16,
                bnb_8bit_use_double_quant=True,
                bnb_8bit_quant_type='nf8')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\rnltl\anaconda3\envs\gemma\lib\site-packages\transformers\quantizers\auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
S

In [3]:
enable_btn = gr.Button(interactive=True)
disable_btn = gr.Button(interactive=False)
spell_chat, potion_chat, other_chat, house_chat = False, False, False, False

question_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def show_image(path):
    # Convert To PIL Image
    image = Image.open(path)
    return image

def spell_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    spell_chat, potion_chat, other_chat, house_chat = True, False, False, False
    bot_message = "Welcome to Magic Spell Class!\nTell me what you want to achieve, and I’ll suggest the perfect spell for it!\nIf you ask in the format: 'What spell can I use when I ~?', I can give you even better suggestions!"
    txt_box = gr.Textbox(value="What spell can I use when I ", interactive=True)
    prof_IMG = show_image("./IMG/Spell_stand.jpg")
    chat_history.append([None, bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def potion_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    spell_chat, potion_chat, other_chat, house_chat = False, True, False, False
    bot_message = "Welcome to Potion Class!\nTell me what you want to achieve, and I’ll suggest the perfect potion for it!\nIf you ask in the format: 'What potion can I make when I ~?', I can give you even better suggestions!"
    txt_box = gr.Textbox(value="What potion can I make when I ", interactive=True)
    prof_IMG = show_image("./IMG/Potion_stand.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def other_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    spell_chat, potion_chat, other_chat, house_chat = False, False, True, False
    bot_message = "Welcome to Library!\nWelcome to the library! Feel free to ask me anything if you're curious!"
    txt_box = gr.Textbox(placeholder="Ask anything you're curious about in the Wizarding World!", value="", interactive=True)
    prof_IMG = show_image("./IMG/Other_stand.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def house_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    spell_chat, potion_chat, other_chat, house_chat = False, False, False, True
    bot_message = "Hmm, let's see… where shall I place you?"
    txt_box = gr.Textbox(placeholder="faiosdjfopasdjfoasdjfopsd", value="", interactive=True)
    prof_IMG = show_image("./IMG/sorting_hat.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def ask_question(chat_history, text_data, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    if spell_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/Spell_think.jpg")
        chat_history.append([text_data, None])
        text_data = gr.Textbox(value="What spell can I use when I ", interactive=True)
        return chat_history, prof_IMG, text_data
    
    if potion_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/Potion_think.jpg")
        chat_history.append([text_data, None])
        text_data = gr.Textbox(value="What potion can I make when I ", interactive=True)
        return chat_history, prof_IMG, text_data
    
    if other_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/Other_think.jpg")
        chat_history.append([text_data, None])
        text_data = gr.Textbox(placeholder="Ask anything you're curious about in the Wizarding World!", value="", interactive=True)
        return chat_history, prof_IMG, text_data
    
    if house_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/sorting_hat.jpg")
        chat_history.append(["test message", None])
        text_data = gr.Textbox(value="asdfasdfasdfasdf", interactive=True)
        return chat_history, prof_IMG, text_data

def clean_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    if spell_chat:
        bot_message = "Welcome to Magic Spell Class!\nTell me what you want to achieve, and I’ll suggest the perfect spell for it!\nIf you ask in the format: 'What spell can I use when I ~?', I can give you even better suggestions!"
        prof_IMG = show_image("./IMG/Spell_stand.jpg")
    if potion_chat:
        bot_message = "Welcome to Potion Class!\nTell me what you want to achieve, and I’ll suggest the perfect potion for it!\nIf you ask in the format: 'What potion can I make when I ~?', I can give you even better suggestions!"
        prof_IMG = show_image("./IMG/Potion_stand.jpg")
    if other_chat:
        bot_message = "Welcome to Library!\nWelcome to the library! Feel free to ask me anything if you're curious!"
        prof_IMG = show_image("./IMG/Other_stand.jpg")
    if house_chat:
        bot_message = "Hmm, let's see… where shall I place you?"
        prof_IMG = show_image("./IMG/sorting_hat.jpg")
    chat_history.append([None, bot_message])
    return chat_history, prof_IMG

def run_model(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""
    
    inputs = HP_chatbot.tokenizer(
    [
        alpaca_prompt.format(
            chat_history[-1][0], # instruction
            "", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    output = HP_chatbot.generate_answer(inputs)
    chat_history[-1][1] = output
    return chat_history

def end_chatting(chat_history, request: gr.Request):
    chat_history.clear()
    chat_history.append([None,"Welcome to Hogwarts!"])
    return chat_history, show_image("./IMG/HogwartGemma.jpg")

def result_img(request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    if spell_chat:
        return show_image("./IMG/Spell_show.jpg")
    if potion_chat:
        return show_image("./IMG/Potion_show.jpg")
    if other_chat:
        return show_image("./IMG/Other_show.jpg")
    if house_chat:
        return show_image("./IMG/sorting_hat.jpg")

def build_gradio(concurrency_count=10):
    textbox = gr.Textbox(show_label=False, placeholder="Welcome to Hogwart!", container=False, interactive = False)
    with gr.Blocks(
            theme='HaleyCH/HaleyCH_Theme',
        ) as demo:
        state = gr.State()
        with gr.Row():
            with gr.Column(scale=3):

                # imagebox = gr.Image(interactive=False)
                imagebox = gr.Image(value=show_image("./IMG/HogwartGemma.jpg"), type="pil", interactive=False, show_label=False, show_download_button=False, show_fullscreen_button=False)
                # image_process_mode = gr.Radio(
                #     ["Crop", "Resize", "Pad", "Default"],
                #     value="Default",
                #     label="Preprocess for non-square image", visible=False)
                
                spell_btn = gr.Button(icon="./IMG/wand_icon.png", value="Spell", interactive = True, scale = 2)
                potion_btn = gr.Button(icon="./IMG/cauldron.png", value="Potion", interactive = True, scale = 2)
                other_btn = gr.Button(icon="./IMG/golden-snitch.png", value="Others", interactive = True, scale = 2)
                house_btn = gr.Button(icon="./IMG/hat_icon.png", value="Find your House", interactive = True, scale = 2)

            with gr.Column(scale=8):
                initial_message = [None,"Welcome to Hogwarts!"]
                chatbot = gr.Chatbot(
                    label='Free to ask!',
                    value= [initial_message],
                    height=600,
                )
                with gr.Row():
                    with gr.Column(scale=8):
                        textbox.render()
                    with gr.Column(scale=1, min_width=50):
                        submit_btn = gr.Button(value="Send", variant="primary", interactive = False)

                with gr.Row(elem_id="buttons") as button_row:
                    finish_btn = gr.Button(value="🏁 End Talking", interactive = False)
                    clear_btn = gr.Button(value="🗑️  Clear", interactive=False)    
        box_list = [spell_btn, potion_btn, other_btn, house_btn, textbox, submit_btn, finish_btn, clear_btn]
        spell_btn.click(
            spell_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        potion_btn.click(
            potion_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        other_btn.click(
            other_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        house_btn.click(
            house_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        submit_btn.click(
            ask_question,
            inputs = [chatbot, textbox],
            outputs = [chatbot, imagebox, textbox]
        ).then(
            run_model,
            inputs = [chatbot],
            outputs = [chatbot] 
        ).then(
            result_img,
            outputs=[imagebox]
        )
        finish_btn.click(
            end_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox]
        )
        clear_btn.click(
            clean_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox]
        )

    return demo

import argparse
if __name__ == "__main__":

    # 컨트롤러에서 사용가능한 모델 가져옴. 나는 모델 하나만 쓸 것이기 때문에, get_model_list()에서 해당 모델이 동작하고 있는 url을 넘겨주면 된다!
    # models = [args.model_url] 
    
    # HP_chatbot = HPChatBot(load_in_8bit=True,
    #                    bnb_8bit_compute_dtype=torch.float16,
    #                    bnb_8bit_use_double_quant=True,
    #                    bnb_8bit_quant_type='nf8')
    
    # Gradio를 이용해서 데모 만들기
    demo = build_gradio()
    demo.queue(
        api_open=False
    ).launch(
        server_port=7860,
        share=True
    )
    


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://9f25218e53195d04c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:
# demo.close()